### 1 - Import libs

**ONLY EXECUTE THIS CELL**

In [1]:
import time 
from pathlib import Path
from datetime import datetime as dt

import ee
from sepal_ui.mapping import SepalMap
import geemap
# initialize EE    
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

import helpers as h

installed = !pip freeze | grep "retry=="
if not installed:
    !pip install --user retry

Styles()

/usr/local/lib/python3.8/dist-packages/bfast/__init__.py:10: UserWarning: cupy is not available in this environment, GPU fonctionnalities won't be available
  warn("cupy is not available in this environment, GPU fonctionnalities won't be available")


### 2 - Basic Input Variables

**FILL IN YOUR INPUTS**

In [2]:
# Area of Interest
import ee
import geemap

# country case example
#country = 'Kenya'   
#aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', country)) # here any feature collection can be selected

aoi = ee.FeatureCollection("users/Erith7/Sucumbios")

# Point Feature Collection from where to extract points
#fc = ee.FeatureCollection("my_featurecollection_path").filterBounds(aoi)
#point_id_name = "PLOTID"  # the column/property within your point feature collection from which to take the UNIQUE point id

#fc = ee.FeatureCollection("users/impressfao/centre_2k_grid").filterBounds(aoi)
#fc = ee.FeatureCollection("users/Erith7/PuntosSucumbios").filterBounds(aoi)

#point_id_name = "bb_tileID"
#point_id_name = "Index"
point_id_name = "point_id"
#print(fc.size().getInfo())

grid_size = 0.01  # that's the size of the grid we are parallelizing on
workers = 10 # number of parallel EE requests

# Time of interest (onsidering the historical period)
start_date = "2012-01-01"
end_date = "2021-12-31"

# Monitoring period
start_monitor = "2016-01-01"

# Directory where output and temp files will go
outdir = 'resultsMAATE'

# Select algorithms to run
cusum_deforest = True
bfast_monitor = True
bs_slope = True
ts_metrics = True
ccdc = True
global_products = False

band='ndvi' # other choices: ndfi, ndmi, mndwi

### DO NOT CHANGE YET ###
satellite='Landsat'

In [3]:
# Grid Size (in degrees)
#grid_size = 0.1
# Generate the grid
grid, grid_fc = h.generate_grid(aoi, grid_size, grid_size, 0, 0)

# get a random point over each grid cell
randomInCell = grid_fc.map(h.get_random_point).filterBounds(aoi)

# get the centroid of each gridcell
centerInCell = grid_fc.map(h.get_center_point).filterBounds(aoi)

# stratified sampling based on ESA LC
imageCollection = ee.ImageCollection("ESA/WorldCover/v100")
esaLc = imageCollection.filterBounds(aoi).first().clip(aoi)

stratifiedSamples = esaLc.stratifiedSample(**{
    "numPoints": 100,
    "region": aoi,
    "scale": 100, 
    "seed": 42, 
    "tileScale": 4, 
    "geometries": True
}).map(h.set_id)

In [4]:
# intialize Map
Map = SepalMap(['HYBRID'])
Map.zoom_ee_object(aoi.geometry())

# add layers
Map.addLayer(ee.Image().byte().paint(featureCollection=aoi,color=1,width=3), {"palette": "blue"}, 'Area of interest')
Map.addLayer(esaLc, False, 'ESA Worldcover 2020')
Map.addLayer(stratifiedSamples.draw(color="black", pointRadius=2), {}, 'Stratified Samples per class from ESA LC')
Map.addLayer(ee.Image().byte().paint(featureCollection=grid,color=1,width=3), {"palette": "black"}, 'Grid')
Map.addLayer(randomInCell.draw(color="purple", pointRadius=2), {}, '1 Center Sample per gridcell (Systematic)')
Map.addLayer(centerInCell.draw(color="white", pointRadius=2), {}, '1 Random Sample per gridcell (Systematic random)')
#
Map

SepalMap(center=[-0.041799613075468256, -77.15800826163783], controls=(ZoomControl(options=['position', 'zoom_…

### 3- Algorithm parameter settings

**Edit for advanced users, otherwise just execute**

In [6]:
# bfast parameters
bfast_params = {
    'run': bfast_monitor,
    'start_monitor': start_monitor, 
    'freq': 365,
    'k': 3, 
    'hfrac':0.25, 
    'trend': True, 
    'level':0.05, 
    'backend':'python'}

cusum_params = {
    'run': cusum_deforest,
    'nr_of_bootstraps': 1000
}

bs_slope_params = {
    'run': bs_slope,
    'nr_of_bootstraps': 1000
}

ts_metrics_params = {
    'run': ts_metrics,
    'outlier_removal': True,
    'z_threshhold': 3
}

ccdc_params = {
    'run': ccdc,
}

global_products = {
    'run': global_products,
}

### DO NOT CHANGE ###
### GATHER ALL INFO INTO A DICT #####
config_dict = {
    'work_dir': outdir,
    'workers': 10,
    'ts_params': {
        'start_date': start_date,
        'start_monitor': start_monitor,
        'end_date': end_date,
        'point_id': point_id_name,
        'grid_size': grid_size,
        'band': band,
        'satellite': satellite
    },
    'bfast_params': bfast_params,
    'cusum_params': cusum_params,
    'bs_slope_params': bs_slope_params,
    'ts_metrics_params': ts_metrics_params,
    'ccdc_params': ccdc_params,
    'global_products': global_products
}

# Run the whole thing

**Execute only**

In [7]:
h.get_change_data(aoi, centerInCell, config_dict)

 You changed grid size. For this all temporary results are now being deleted.
 Parallelizing time-series extraction on 10 threads for a total of 397 grid cells.
 Grid cell 0 already has been extracted. Going on with next grid cell.
 Grid cell 1 already has been extracted. Going on with next grid cell.
 Processing gridcell 3
 Processing gridcell 2
 Processing gridcell 11
 Processing gridcell 7
 Processing gridcell 5
 Processing gridcell 4
 Processing gridcell 10
 Processing gridcell 6
 Processing gridcell 9
 Processing gridcell 8


/usr/local/lib/python3.8/dist-packages/geopandas/array.py:151: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.from_shapely(data), crs=crs)
/usr/local/lib/python3.8/dist-packages/geopandas/array.py:151: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.from_shapely(data), crs=crs)
2022-06-08 14:54:07.345110: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-08 14:54:07.345158: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (3ae5da5bebb3): /proc/driver/nvidia/version does not exist
2022-06-08 14:54:07.441042: I tensorflow/core/platform/cpu_feature_guard.cc:151] This

 Grid cell 5 with 1 points done in: 0:04:26.784735
 Grid cell 3 with 1 points done in: 0:04:27.449131
 Grid cell 6 with 1 points done in: 0:04:27.880811
 Grid cell 4 with 1 points done in: 0:04:28.221352
 Grid cell 9 with 1 points done in: 0:04:28.549298
 Grid cell 7 with 1 points done in: 0:04:28.842338
 Grid cell 10 with 1 points done in: 0:04:29.432495
 Processing gridcell 14
 Processing gridcell 18
 Processing gridcell 17
 Processing gridcell 16
 Processing gridcell 15
 Processing gridcell 12
 Processing gridcell 13
 Grid cell 8 with 1 points done in: 0:04:55.093993
 Grid cell 2 with 1 points done in: 0:04:58.198804
 Grid cell 11 with 1 points done in: 0:05:02.003995
 Processing gridcell 20
 Processing gridcell 19
 Processing gridcell 21
 Grid cell 16 with 1 points done in: 0:03:20.748981
 Processing gridcell 22
 Grid cell 13 with 1 points done in: 0:03:45.520569
 Processing gridcell 23
 Grid cell 15 with 1 points done in: 0:04:03.002021
 Grid cell 20 with 1 points done in: 0:03:35

 Processing gridcell 119
 Grid cell 109 with 1 points done in: 0:04:18.546467
 Processing gridcell 120
 Grid cell 111 with 1 points done in: 0:04:10.666909
 Processing gridcell 121
 Grid cell 112 with 1 points done in: 0:04:32.943149
 Processing gridcell 122
 Grid cell 113 with 1 points done in: 0:04:45.389009
 Processing gridcell 123
 Grid cell 114 with 1 points done in: 0:04:49.612874
 Grid cell 116 with 1 points done in: 0:04:19.051132
 Processing gridcell 124
 Processing gridcell 125
 Grid cell 115 with 1 points done in: 0:04:40.797304
 Grid cell 117 with 1 points done in: 0:04:32.106881
 Processing gridcell 126
 Processing gridcell 127
 Grid cell 119 with 1 points done in: 0:04:11.562493
 Grid cell 118 with 1 points done in: 0:04:25.407790
 Processing gridcell 128
 Processing gridcell 129
 Grid cell 121 with 1 points done in: 0:04:16.741217
 Processing gridcell 130
 Grid cell 120 with 1 points done in: 0:05:00.231366
 Processing gridcell 131
 Grid cell 122 with 1 points done in: 0

 Processing gridcell 223
 Processing gridcell 225
 Processing gridcell 224
 Grid cell 214 with 1 points done in: 0:06:11.187778
 Grid cell 215 with 1 points done in: 0:05:28.604342
 Processing gridcell 227
 Processing gridcell 226
 Grid cell 219 with 1 points done in: 0:03:58.606860
 Processing gridcell 228
 Grid cell 218 with 1 points done in: 0:05:43.738257
 Grid cell 220 with 1 points done in: 0:03:59.955574
 Processing gridcell 230
 Processing gridcell 229
 Grid cell 221 with 1 points done in: 0:05:22.356935
 Processing gridcell 231
 Grid cell 224 with 1 points done in: 0:04:33.131847
 Grid cell 227 with 1 points done in: 0:04:02.946223
 Grid cell 225 with 1 points done in: 0:04:35.435654
 Processing gridcell 233
 Processing gridcell 234
 Processing gridcell 232
 Grid cell 223 with 1 points done in: 0:05:45.909477
 Grid cell 222 with 1 points done in: 0:06:01.334347
 Processing gridcell 236
 Processing gridcell 235
 Grid cell 226 with 1 points done in: 0:05:40.085547
 Processing gr

/usr/local/lib/python3.8/dist-packages/bfast/monitor/python/base.py:255: RuntimeWarning: invalid value encountered in sqrt
  sigma = np.sqrt(np.sum(y_error[:ns] ** 2) / (ns - (2 + 2 * self.k)))


 Grid cell 233 with 1 points done in: 0:04:20.333458
 Processing gridcell 242
 Grid cell 232 with 1 points done in: 0:04:31.067006
 Processing gridcell 243
 Grid cell 234 with 1 points done in: 0:05:03.471778
 Processing gridcell 244
 Grid cell 235 with 1 points done in: 0:04:32.411965
 Grid cell 236 with 1 points done in: 0:04:37.307857
 Processing gridcell 245
 Processing gridcell 246
 Grid cell 237 with 1 points done in: 0:04:46.189182
 Processing gridcell 247
 Grid cell 238 with 1 points done in: 0:04:59.128988
 Processing gridcell 248
 Grid cell 239 with 1 points done in: 0:05:02.993490
 Processing gridcell 249
 Grid cell 240 with 1 points done in: 0:05:14.704981
 Processing gridcell 250
 Grid cell 242 with 1 points done in: 0:04:08.130835
 Processing gridcell 251
 Grid cell 241 with 1 points done in: 0:05:58.865025
 Processing gridcell 252
 Grid cell 243 with 1 points done in: 0:05:56.221790
 Grid cell 244 with 1 points done in: 0:05:26.006060
 Processing gridcell 253
 Processing

 Processing gridcell 347
 Grid cell 340 with 1 points done in: 0:05:00.475302
 Processing gridcell 348
 Grid cell 341 with 1 points done in: 0:04:57.847367
 Grid cell 339 with 1 points done in: 0:05:19.477305
 Grid cell 337 with 1 points done in: 0:05:28.715852
 Grid cell 338 with 1 points done in: 0:05:27.357345
 Processing gridcell 349
 Processing gridcell 350
 Processing gridcell 352
 Processing gridcell 351
 Grid cell 343 with 1 points done in: 0:04:58.151891
 Grid cell 344 with 1 points done in: 0:04:59.454619
 Processing gridcell 353
 Processing gridcell 354
 Grid cell 345 with 1 points done in: 0:04:36.215234
 Processing gridcell 355
 Grid cell 346 with 1 points done in: 0:04:15.541007
 Processing gridcell 356
 Grid cell 347 with 1 points done in: 0:04:29.911694
 Processing gridcell 357
 Grid cell 350 with 1 points done in: 0:04:19.581065
 Grid cell 348 with 1 points done in: 0:04:33.339301
 Processing gridcell 358
 Processing gridcell 359
 Grid cell 351 with 1 points done in: 0

/usr/local/lib/python3.8/dist-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


 Processing has been finished successfully. Check for final_results files in your output directory.
